In [11]:
# Support for math
import numpy as np
import math

# Plotting tools
from matplotlib import pyplot as plt
import matplotlib
from matplotlib import cm
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

#File Tools for local
import pandas as pd
import sys

#Sampling
from pyDOE import lhs
from scipy.interpolate import griddata
from smt.sampling_methods import LHS
import random

import torch
from botorch.models.gp_regression import HeteroskedasticSingleTaskGP
from botorch.models.gp_regression import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils.transforms import normalize
from gpytorch.mlls import ExactMarginalLogLikelihood


In [12]:
# Import all data
data_r1 = pd.read_csv('/Users/ctuwsunlab/Documents/GitHub/PNNL-ML_for_Organic_Flow_Battery_Materials/Round1/extracted_data_round1.csv')
data_r2B = pd.read_csv('/Users/ctuwsunlab/Documents/GitHub/PNNL-ML_for_Organic_Flow_Battery_Materials/Round2/ModelB/extracted_data_round2B.csv')
data_r3B = pd.read_csv('/Users/ctuwsunlab/Documents/GitHub/PNNL-ML_for_Organic_Flow_Battery_Materials/Round3/ModelB/extracted_data_round3B.csv')
data_redo =pd.read_csv('/Users/ctuwsunlab/Documents/GitHub/PNNL-ML_for_Organic_Flow_Battery_Materials/Round_Redo/extracted_data_round_redo.csv')

raw_data01 = data_r1[['01_time','01_temp','01_sulf','01_anly','01_yield product']].rename(columns = {'01_time':"time",'01_temp':"temp",'01_sulf':"sulf",'01_anly':"anly",'01_yield product':"yield product"})
raw_data2B = data_r2B[['2B_time','2B_temp','2B_sulf','2B_anly','2B_yield product']].rename(columns = {'2B_time':"time",'2B_temp':"temp",'2B_sulf':"sulf",'2B_anly':"anly",'2B_yield product':"yield product"})
raw_data3B = data_r3B[['3B_time','3B_temp','3B_sulf','3B_anly','3B_yield product']].rename(columns = {'3B_time':"time",'3B_temp':"temp",'3B_sulf':"sulf",'3B_anly':"anly",'3B_yield product':"yield product"})
raw_dataredo = data_redo[['model ID','time','temp','sulf','anly','yield product']]

# Deviec set up
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32


In [13]:
avg_data01 = raw_data01.rolling(window=3).mean().iloc[2::3].reset_index(drop=True)[['time','temp','sulf','anly','yield product']]
std_data01 = raw_data01 .rolling(window=3).std().iloc[2::3].reset_index(drop=True)[['yield product']].rename(columns = {'yield product':"var yield"})
data01 = pd.concat([avg_data01, std_data01], axis=1)

avg_data2B = raw_data2B .rolling(window=3).mean().iloc[2::3].reset_index(drop=True)[['time','temp','sulf','anly','yield product']]
std_data2B = raw_data2B .rolling(window=3).std().iloc[2::3].reset_index(drop=True)[['yield product']].rename(columns = {'yield product':"var yield"})
data2B = pd.concat([avg_data2B, std_data2B], axis=1)

avg_data3B = raw_data3B.rolling(window=3).mean().iloc[2::3].reset_index(drop=True)[['time','temp','sulf','anly','yield product']]
std_data3B = raw_data3B.rolling(window=3).std().iloc[2::3].reset_index(drop=True)[['yield product']].rename(columns = {'yield product':"var yield"})
data3B = pd.concat([avg_data3B, std_data3B], axis=1)

avg_data_redo = raw_dataredo.rolling(window=3).mean().iloc[2::3].reset_index(drop=True)[['model ID','time','temp','sulf','anly','yield product']]
std_data_redo = raw_dataredo.rolling(window=3).std().iloc[2::3].reset_index(drop=True)[['yield product']].rename(columns = {'yield product':"var yield"})
redo_data = pd.concat([avg_data_redo, std_data_redo], axis=1)

# Add a category column to each dataframe before concatenating
data01['category'] = 'R 01'
data2B['category'] = 'R 2B'
data3B['category'] = 'R 3B'
redo_data['category'] = 'redo'

# Concatenate the dataframes
modelB_df = pd.concat([data01, data2B, data3B], axis=0, ignore_index=True)
outlier_ID = [78, 80, 88, 89] 

# Replace outliers in modelB_df with their corresponding model IDs in redo_data
for outlier_id in outlier_ID:
    # Find the corresponding row in redo_data
    replacement_row = redo_data[redo_data['model ID'] == outlier_id]
    if not replacement_row.empty:
        # Replace the row in modelB_df
        modelB_df.loc[modelB_df['time'] == replacement_row['time'].values[0], 
                      ['time', 'temp', 'sulf', 'anly', 'yield product', 'var yield', 'category']] = replacement_row[['time', 'temp', 'sulf', 'anly', 'yield product', 'var yield', 'category']].values


# Get high yield samples
high_yield_df = modelB_df[modelB_df['yield product'] > 0.9]
high_yield_df 

data3B = modelB_df[(modelB_df['category'] == 'R 3B').idxmax():(modelB_df['category'] == 'R 3B').idxmax()+15].reset_index(drop=True)

# Model Sets
set_dataB = pd.concat([data01, data2B, data3B], axis=0, ignore_index=True)


In [ ]:
# Normalize the data
bounds = torch.tensor([[30., 20., 75. , 33.], [600., 170., 100.,100.]], dtype=torch.float32) 
# Define all combinations of variables for triple pair plots
variable_combinations = [('time', 'temp', 'sulf'), ('time', 'temp', 'anly'), ('time', 'sulf', 'anly'),('temp', 'sulf', 'anly'),('sulf', 'anly', 'time')]
fixed_variable = ['anly', 'sulf', 'temp', 'time']
#variable_combinations = [('time', 'temp', 'sulf', 'anly'), ('time', 'sulf', 'anly','temp'), ('temp', 'sulf', 'anly','time'), ('time', 'temp', 'anly','sulf')]

def data_prep(df,x1,x2,x3,x4,bounds):
    x1 = torch.tensor([df[x1]], dtype=torch.float32).T
    x2 = torch.tensor([df[x2]], dtype=torch.float32).T
    x3 = torch.tensor([df[x3]], dtype=torch.float32).T
    x4 = torch.tensor([df[x4]], dtype=torch.float32).T

    train_x = torch.hstack([x1, x2, x3, x4])
    train_y = torch.tensor(df['yield product'], dtype=torch.float32).reshape(-1, 1)
    train_yvar = torch.tensor(df['var yield'], dtype=torch.float32).reshape(-1, 1)

    norm_x = normalize(train_x, bounds)

    return norm_x, train_y, train_yvar

# Function to generate input data based on variable combination
def generate_input_data(A, B, c, d, combination):
    if combination == ('time', 'temp', 'sulf'):
        return torch.tensor(np.array([[A[i, j], B[i, j], c, d] for i in range(A.shape[0]) for j in range(A.shape[1])]), dtype=dtype)
    elif combination == ('time', 'temp', 'anly'):
        return torch.tensor(np.array([[A[i, j], B[i, j], d, c] for i in range(A.shape[0]) for j in range(A.shape[1])]), dtype=dtype)
    elif combination == ('time', 'sulf', 'anly'):
        return torch.tensor(np.array([[A[i, j], d, B[i, j], c ] for i in range(A.shape[0]) for j in range(A.shape[1])]), dtype=dtype)
    elif combination == ('temp', 'sulf', 'anly'):
        return torch.tensor(np.array([[d, A[i, j], B[i, j], c] for i in range(A.shape[0]) for j in range(A.shape[1])]), dtype=dtype)
    elif combination == ('time', 'sulf', 'temp'):
        return torch.tensor(np.array([[A[i, j], c,  B[i, j], d] for i in range(A.shape[0]) for j in range(A.shape[1])]), dtype=dtype)
    elif combination == ('sulf', 'anly', 'time'):
        return torch.tensor(np.array([[c, d,  A[i, j], B[i,j]] for i in range(A.shape[0]) for j in range(A.shape[1])]), dtype=dtype)

def gp_4d(train_x, train_y, train_yvar, test_x):
    # Define the initial GP model
    gp = HeteroskedasticSingleTaskGP(train_X=train_x, train_Y=train_y, train_Yvar=train_yvar) 
    #gp = SingleTaskGP(train_X=train_x, train_Y=train_y, train_Yvar=train_yvar, covar_module=kernel) 
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    # Get posterior predictions
    gp.eval()
    with torch.no_grad():
        posterior = gp.posterior(test_x)

    mean = posterior.mean.squeeze().numpy()
    lower, upper = posterior.mvn.confidence_region()
    var = posterior.variance.squeeze().numpy()

    return mean, var

def create_sclices(norm_x, out_y, noise_y,A,B,c,d_fixed,combination):
    mean_values = []
    var_values = []
    for c in c_slices:
        input_data = generate_input_data(A, B, c, d_fixed, combination)
        mean, var = gp_4d(norm_x, out_y, noise_y, input_data)
        mean_values.append(mean.reshape(A.shape))  # Reshape to match the grid
        var_values.append(var.reshape(A.shape))  # Reshape to match the grid

    return mean_values, var_values

def slcied_plotting(mean_vals1,mean_vals2,mean_vals3,mean_vals4,mean_vals5,combination,minmax,colormap):
    global_min = minmax[0]
    global_max = minmax[1]
    # Create a new figure with subplots for each combination
    fig = make_subplots(rows=1, cols=5, subplot_titles=('temp: 0', 'temp: 0.25','temp: 0.5', 'temp: 0.75','temp: 1.0'),
                    specs=[[{'type': 'surface'}, {'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]])

    for i, (c, y_grid1, y_grid2, y_grid3, y_grid4, y_grid5) in enumerate(zip(c_slices, mean_vals1,mean_vals2,mean_vals3,mean_vals4,mean_vals5), start=1):
        fig.add_trace(go.Surface(
            x=A,
            y=B,
            z=c * np.ones_like(A),  # Z-coordinate for slicing
            surfacecolor=y_grid1,  # Use predicted `y` as contour
            colorscale=colormap,
            cmin=global_min,
            cmax=global_max,
            showscale=True if i == 1 else False,  # Show color scale only on the first slice
            #   colorbar_x=0.45,
            opacity=0.7
        ), row=1, col=1)
  
        fig.add_trace(go.Surface(
            x=A,
            y=B,
            z=c * np.ones_like(A),  # Z-coordinate for slicing
            surfacecolor=y_grid2,  # Use predictediance as contour
            cmin=global_min,
            cmax=global_max,
            colorscale=colormap,
            showscale=True if i == 1 else False,  # Show color scale only on the first slice
            #colorbar_x=0.45,
            opacity=0.7
        ), row=1, col=2)

        fig.add_trace(go.Surface(
            x=A,
            y=B,
            z=c * np.ones_like(A),  # Z-coordinate for slicing
            surfacecolor=y_grid3,  # Use predictediance as contour
            cmin=global_min,
            cmax=global_max,
            colorscale=colormap,
            showscale=True if i == 1 else False,  # Show color scale only on the first slice
            opacity=0.7
        ), row=1, col=3)

        fig.add_trace(go.Surface(
            x=A,
            y=B,
            z=c * np.ones_like(A),  # Z-coordinate for slicing
            surfacecolor=y_grid4,  # Use predictediance as contour
            cmin=global_min,
            cmax=global_max,
            colorscale=colormap,
            showscale=True if i == 1 else False,  # Show color scale only on the first slice
            opacity=0.7
        ), row=1, col=4)

        fig.add_trace(go.Surface(
            x=A,
            y=B,
            z=c * np.ones_like(A),  # Z-coordinate for slicing
            surfacecolor=y_grid5,  # Use predictediance as contour
            cmin=global_min,
            cmax=global_max,
            colorscale=colormap,
            showscale=True if i == 1 else False,  # Show color scale only on the first slice
            opacity=0.7
        ), row=1, col=5)
        

    fig.update_layout(
        height=400,
        width=1300,
        margin=dict(l=50, r=50, b=50, t=50),
        scene=dict(
            xaxis_title=combination[0],
            yaxis_title=combination[1],
            zaxis_title=combination[2]
        ),
        scene2=dict(
            xaxis_title=combination[0],
            yaxis_title=combination[1],
            zaxis_title=combination[2]
        ),
        scene3=dict(
            xaxis_title=combination[0],
            yaxis_title=combination[1],
            zaxis_title=combination[2]
        ),
        scene4=dict(
            xaxis_title=combination[0],
            yaxis_title=combination[1],
            zaxis_title=combination[2]
        ),
        scene5=dict(
            xaxis_title=combination[0],
            yaxis_title=combination[1],
            zaxis_title=combination[2]
        )
    )

    fig.show()

comb_id = 3

In [14]:
# Generate a grid for demonstration (adjust as needed for your data range)
num_points = 15  # Points per axis for plotting
a = np.linspace(0, 1, num_points)
b = np.linspace(0, 1, num_points)
c_slices = np.linspace(0, 1, 12)  # Slicing variable
d_fixed =  0.5 # Example fixed value for d - three temperatures

# # Create meshgrid for a, b
A, B = np.meshgrid(a, b)

In [ ]:
# Create the training data
norm_0, out_0, noise_0 = data_prep(data00,'time','temp','sulf','anly',bounds)

mean_vals1, var_vals1 = create_sclices(norm_0, out_0, noise_0, A,B,c_slices,0.0 ,variable_combinations[comb_id]) # fixed variable: temp = 0.0
mean_vals2, var_vals2 = create_sclices(norm_0, out_0, noise_0, A,B,c_slices,0.25,variable_combinations[comb_id]) # fixed variable: temp = 0.25
mean_vals3, var_vals3 = create_sclices(norm_0, out_0, noise_0, A,B,c_slices,0.5 ,variable_combinations[comb_id]) # fixed variable: temp = 0.50
mean_vals4, var_vals4 = create_sclices(norm_0, out_0, noise_0, A,B,c_slices,0.75,variable_combinations[comb_id]) # fixed variable: temp = 0.75
mean_vals5, var_vals5 = create_sclices(norm_0, out_0, noise_0, A,B,c_slices,1.0 ,variable_combinations[comb_id]) # fixed variable: temp = 1.0


slcied_plotting(mean_vals1,mean_vals2,mean_vals3,mean_vals4,mean_vals5,variable_combinations[comb_id],[0,1],'Plasma')


In [ ]:
set_data2B = pd.concat([data01, data2B ], ignore_index=True)
norm_2B, out_2B, noise_2B = data_prep(set_data2B,'time','temp','sulf','anly',bounds)

mean_vals1, var_vals1 = create_sclices(norm_2B, out_2B, noise_2B, A,B,c_slices,0.0 ,variable_combinations[comb_id]) # fixed variable: temp = 0.00
mean_vals2, var_vals2 = create_sclices(norm_2B, out_2B, noise_2B, A,B,c_slices,0.25,variable_combinations[comb_id]) # fixed variable: temp = 0.25
mean_vals3, var_vals3 = create_sclices(norm_2B, out_2B, noise_2B, A,B,c_slices,0.5 ,variable_combinations[comb_id]) # fixed variable: temp = 0.50
mean_vals4, var_vals4 = create_sclices(norm_2B, out_2B, noise_2B, A,B,c_slices,0.75,variable_combinations[comb_id]) # fixed variable: temp = 0.75
mean_vals5, var_vals5 = create_sclices(norm_2B, out_2B, noise_2B, A,B,c_slices,1.0 ,variable_combinations[comb_id]) # fixed variable: temp = 1.00

slcied_plotting(mean_vals1,mean_vals2,mean_vals3,mean_vals4,mean_vals5,variable_combinations[comb_id],[0,1],'Plasma')

abs_min = min([np.min(var_vals1),np.min(var_vals2),np.min(var_vals3),np.min(var_vals4),np.min(var_vals5)])
abs_max = max([np.max(var_vals1),np.max(var_vals2),np.max(var_vals3),np.max(var_vals4),np.max(var_vals5)])
slcied_plotting(var_vals1,var_vals2,var_vals3,var_vals4,var_vals5,variable_combinations[comb_id],[abs_min,abs_max],'Plasma_r')

In [ ]:
set_data3B = pd.concat([data01, data2A, data3B], axis=0, ignore_index=True)
norm_3B, out_3B, noise_3B = data_prep(set_data3B,'time','temp','sulf','anly',bounds)
mean_vals1, var_vals1 = create_sclices(norm_3B, out_3B, noise_3B, A,B,c_slices,0.0 ,variable_combinations[comb_id]) # fixed variable: temp = 0.0
mean_vals2, var_vals2 = create_sclices(norm_3B, out_3B, noise_3B, A,B,c_slices,0.25,variable_combinations[comb_id]) # fixed variable: temp = 0.25
mean_vals3, var_vals3 = create_sclices(norm_3B, out_3B, noise_3B, A,B,c_slices,0.5 ,variable_combinations[comb_id]) # fixed variable: temp = 0.50
mean_vals4, var_vals4 = create_sclices(norm_3B, out_3B, noise_3B, A,B,c_slices,0.75,variable_combinations[comb_id]) # fixed variable: temp = 0.75
mean_vals5, var_vals5 = create_sclices(norm_3B, out_3B, noise_3B, A,B,c_slices,1.0 ,variable_combinations[comb_id]) # fixed variable: temp = 1.0

slcied_plotting(mean_vals1,mean_vals2,mean_vals3,mean_vals4,mean_vals5,variable_combinations[comb_id],[0,1],'Plasma')

abs_min = min([np.min(var_vals1),np.min(var_vals2),np.min(var_vals3),np.min(var_vals4),np.min(var_vals5)])
abs_max = max([np.max(var_vals1),np.max(var_vals2),np.max(var_vals3),np.max(var_vals4),np.max(var_vals5)])
slcied_plotting(var_vals1,var_vals2,var_vals3,var_vals4,var_vals5,variable_combinations[comb_id],[abs_min,abs_max],'Plasma_r')